In [1]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

/home/djjin/anaconda3/envs/conda_visual_HPE/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_May__3_18:49:52_PDT_2022
Cuda compilation tools, release 11.7, V11.7.64
Build cuda_11.7.r11.7/compiler.31294372_0
torch:  1.13 ; cuda:  cu117
detectron2: 0.6


In [2]:
import os
import sys
import logging
import argparse
os.environ["DATASET"] = "../datasets"

pth = '/'.join(sys.path[0].split('/')[:-1])
sys.path.insert(0, pth)

from pprint import pprint
import numpy as np
np.random.seed(1)

home_dir = os.path.abspath(os.getcwd()+"/../")
sys.path.append(home_dir)
print(home_dir)

import warnings
warnings.filterwarnings(action='ignore')
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

from hdecoder.BaseModel import BaseModel
from hdecoder import build_model
from utils.distributed import init_distributed
from utils.arguments import load_opt_from_config_files, load_config_dict_to_opt
from utils.misc import MetricLogger

/home/djjin/Mygit/X-Decoder


Invalid MIT-MAGIC-COOKIE-1 key

In [3]:
from utils.arguments import load_vcoco_opt_command, load_vcoco_parser

cmdline_args = load_vcoco_parser()
cmdline_args.conf_files = [os.path.join(home_dir, "configs/hdecoder/vcoco.yaml")]

model_path = '../data/output/test/00205200/default/raw_model_states.pt'
cmdline_args.overrides = ['DONT_LOAD_MODEL', 'false', 'PYLEARN_MODEL', model_path] 

opt = load_vcoco_opt_command(cmdline_args)
opt = init_distributed(opt)

In [4]:
print(opt['base_path'])
print(opt["RESUME"])

../
False


In [5]:
from trainer import HDecoder_Trainer as Trainer
if cmdline_args.user_dir:
    absolute_user_dir = os.path.abspath(cmdline_args.user_dir)
    opt['base_path'] = absolute_user_dir
trainer = Trainer(opt)
trainer.eval()

INFO:trainer.distributed_trainer:Setting SAVE_DIR as data/output/test
INFO:trainer.distributed_trainer:Using CUDA
INFO:trainer.distributed_trainer:Save config file to data/output/test/conf_copy.yaml
INFO:trainer.distributed_trainer:Base learning rate: 0.0001
INFO:trainer.distributed_trainer:Number of GPUs: 1
INFO:trainer.distributed_trainer:Gradient accumulation steps: 1
INFO:trainer.default_trainer:Imported base_dir at base_path ../
INFO:trainer.default_trainer:Pipeline for training: HDecoderPipeline
INFO:trainer.default_trainer:-----------------------------------------------
INFO:trainer.default_trainer:Evaluating model ... 
INFO:base_dir.pipeline.HDecoderPipeline:CDNHOI(
  (backbone): D2FocalNet(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 96, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
      (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0): BasicLayer(
        (bloc

using use_nms_filter:  True


INFO:utils.model:Loaded backbone.layers.0.blocks.0.dw1.bias, Model Shape: torch.Size([96]) <-> Ckpt Shape: torch.Size([96])
INFO:utils.model:Loaded backbone.layers.0.blocks.0.dw1.weight, Model Shape: torch.Size([96, 1, 3, 3]) <-> Ckpt Shape: torch.Size([96, 1, 3, 3])
INFO:utils.model:Loaded backbone.layers.0.blocks.0.dw2.bias, Model Shape: torch.Size([96]) <-> Ckpt Shape: torch.Size([96])
INFO:utils.model:Loaded backbone.layers.0.blocks.0.dw2.weight, Model Shape: torch.Size([96, 1, 3, 3]) <-> Ckpt Shape: torch.Size([96, 1, 3, 3])
INFO:utils.model:Loaded backbone.layers.0.blocks.0.gamma_1, Model Shape: torch.Size([96]) <-> Ckpt Shape: torch.Size([96])
INFO:utils.model:Loaded backbone.layers.0.blocks.0.gamma_2, Model Shape: torch.Size([96]) <-> Ckpt Shape: torch.Size([96])
INFO:utils.model:Loaded backbone.layers.0.blocks.0.mlp.fc1.bias, Model Shape: torch.Size([384]) <-> Ckpt Shape: torch.Size([384])
INFO:utils.model:Loaded backbone.layers.0.blocks.0.mlp.fc1.weight, Model Shape: torch.Si

Evaluation start ...


INFO:detectron2.data.common:Serializing 4946 elements to byte tensors and concatenating them all ...
INFO:detectron2.data.common:Serialized dataset takes 3.69 MiB


loading annotations into memory...
Done (t=1.81s)
creating index...
index created!
loading vcoco annotations...


INFO:base_dir.pipeline.HDecoderPipeline:Inference done 11/2473. Dataloading: 0.0311 s/iter. Inference: 0.4881 s/iter. Eval: 0.0715 s/iter. Total: 0.5907 s/iter. ETA=0:24:14
INFO:base_dir.pipeline.HDecoderPipeline:Inference done 21/2473. Dataloading: 0.0355 s/iter. Inference: 0.4903 s/iter. Eval: 0.0327 s/iter. Total: 0.5586 s/iter. ETA=0:22:49
INFO:base_dir.pipeline.HDecoderPipeline:Inference done 31/2473. Dataloading: 0.0379 s/iter. Inference: 0.4917 s/iter. Eval: 0.0238 s/iter. Total: 0.5536 s/iter. ETA=0:22:31
INFO:base_dir.pipeline.HDecoderPipeline:Inference done 40/2473. Dataloading: 0.0384 s/iter. Inference: 0.4951 s/iter. Eval: 0.0207 s/iter. Total: 0.5543 s/iter. ETA=0:22:28
INFO:base_dir.pipeline.HDecoderPipeline:Inference done 49/2473. Dataloading: 0.0420 s/iter. Inference: 0.4965 s/iter. Eval: 0.0187 s/iter. Total: 0.5574 s/iter. ETA=0:22:31
INFO:base_dir.pipeline.HDecoderPipeline:Inference done 58/2473. Dataloading: 0.0401 s/iter. Inference: 0.4964 s/iter. Eval: 0.0249 s/it

---------Reporting Role AP (%)------------------
               hold-obj: AP = 31.71 (#pos = 3608)
              sit-instr: AP = 39.27 (#pos = 1916)
             ride-instr: AP = 51.76 (#pos = 556)
               look-obj: AP = 27.79 (#pos = 3347)
              hit-instr: AP = 50.30 (#pos = 349)
                hit-obj: AP = 13.79 (#pos = 349)
                eat-obj: AP = 48.91 (#pos = 521)
              eat-instr: AP = 67.07 (#pos = 521)
             jump-instr: AP = 67.53 (#pos = 635)
              lay-instr: AP = 49.58 (#pos = 387)
    talk_on_phone-instr: AP = 16.76 (#pos = 285)
              carry-obj: AP = 27.92 (#pos = 472)
              throw-obj: AP = 20.37 (#pos = 244)
              catch-obj: AP = 22.81 (#pos = 246)
              cut-instr: AP = 14.50 (#pos = 269)
                cut-obj: AP = 53.14 (#pos = 269)
 work_on_computer-instr: AP = 57.38 (#pos = 410)
              ski-instr: AP = 23.67 (#pos = 424)
             surf-instr: AP = 60.20 (#pos = 486)
       skateboard

{'vcoco_val/vcoco': {'AP_scenario_1': 0.41517952}}